# Introduction

In this notebook we will walk you through implementing a simple A* algorithm for path planning.

We have defined an abstract class for a path planning algorithm called `Graph`<sup>[1]</sup>, as well as an abstract class `Node`. Our `main` uses implementations of this interface to show the result of path planning (displaying the path and giving us its cost based on the algorithm's cost function). Your first task should be to take a look at the constructor and members of those two classes (you can ignore the helper function if you wish to).

An example implementation for a planner algorithm has been in `grass_fire.py`. You can try it out:

In [1]:
import grass_fire
grass_fire.main()

Node cost map:
1 1 1 X X X X X X 1
1 1 1 X X X X X X 1
1 1 1 X X X X X X 1
1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1
YO: 32, 50
Total cost-to-go map, initial point at (1, 2)
 3.0  2.0  3.0    ?    ?    ?    ?    ?    ? 12.0
 2.0  1.0  2.0    ?    ?    ?    ?    ?    ? 11.0
 1.0    0  1.0    ?    ?    ?    ?    ?    ? 10.0
 2.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0
 3.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0 10.0
Path from initial point (1, 2) to goal point (9, 0)
   X    X    X    X    X    X    X    X    X   12
   X    X    X    X    X    X    X    X    X   11
   X    0    X    X    X    X    X    X    X   10
   X    1    2    3    4    5    6    7    8    9
   X    X    X    X    X    X    X    X    X    X


However you will quickly realize that this algorithm scales very poorly to problems with bigger state space:

In [3]:
import jupy_main as jm

jm.run_grass_fire({'grid_size': 5, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 10, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 20, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 50, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 100, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 200, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 400, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 500, 'number_obstacles': 0, 'no_render': True})
jm.run_grass_fire({'grid_size': 1000, 'number_obstacles': 0, 'no_render': True})

YO: 25, 25
Path search took 1.2825649992009858ms, final cost: 8.0
YO: 100, 100
Path search took 2.17200099996262ms, final cost: 18.0
YO: 400, 400
Path search took 6.89325999974244ms, final cost: 38.0
YO: 2500, 2500
Path search took 28.44548700068117ms, final cost: 98.0
YO: 10000, 10000
Path search took 104.03992599913181ms, final cost: 198.0
YO: 40000, 40000
Path search took 610.5604269996547ms, final cost: 398.0
YO: 160000, 160000
Path search took 3582.4503040003037ms, final cost: 798.0
YO: 250000, 250000
Path search took 6890.227003999826ms, final cost: 998.0
YO: 1000000, 1000000
Path search took 57111.03616899982ms, final cost: 1998.0


### Question 1: complexity

a. Given that our map is a square, what is the size of the input `n` based on the size of the grid `m`?

**<font color='red'>$n = m^2$ - the input size `n` (the size of the graph) grows quadratically w.r.t `m`</font>**

b. What seems to be the complexity of this implementation? Please note that we are lookin at `O(n)` and *not* `O(m)`.

<font color='red'>**The complexity seems to be $O(n^2 \cdot ln(n))$. To find this out, notice that, for each iteration with an input size $n = m^2$ and runtime $r(n)$:**
    $$
    \frac{r(n)}{n * ln(n)} = K,~\forall n
    $$
**The intuition behind this result is that the complexity is obviously worse than linear ($O(n)$), but better than quadratic ($O(n^2)$).**
</font>

c. Reading the source of this code (file `grass_fire.py`, function `GrassFire::compute_path`), where is the bottleneck? How could this bottleneck be optimized? What would the complexity be without this bottleneck?

**<font color='red'>To make sure we process the nodes with the lowest cost every iteration (if not, we may explore up to $n^3$ if I'm not mistaken), we sort the list of nodes to explore. Sorting a list takes $O(n \cdot ln(n))$ on the general case. Using a binary heap could reduce the insertion + sorting to a $O(1)$ operation.**
</font>
<font color='red'>  
 **Regarding the new complexity: we explore nodes in increasing cost, and the cost-to-go depends on the destination node only. Those two facts mean that when we set the cost of a node, this cost is guaranteed to be minimum. Since nodes are inserted to our heap only when their cost is reduced, and the cost is reduced exactly once for each node (the first time we process them), we add exactly `n` nodes to the list, and process them exactly once. The new complexity is therefore is $O(n)$.</font>**



# Reminder: the algorithm description

TODO: make sure those guys know wassup


# The beginning: our graph's Node class

For this exercise we give you already the definition of the `Node` we will be using. Notice the differences between `AstarNode` and `GrassFireNode`: this is why we are coupling graph creation with graph search!<sup>[1]</sup>

In [16]:
from typing import Optional
from graph import Node
import heapq


class AstarNode(Node):
    def __init__(self, x: int, y: int, cost_to_go: float) -> None:
        super().__init__(x, y, cost_to_go)
        self.h = 0.0
        self.parent: Optional[AstarNode] = None

    @property
    def f(self):
        return self.h + self.cost
    
    def __lt__(self, other):
        return self.f < other.f if self.f != other.f else self.h < other.h

The `x`, `y`, and `cost_to_go` are passed to the base class. `h` is initialized to some placeholder value (could have chosen `inf` as well). The `cost` attribute of the base `Node` corresponds to the `g` function in A* parlance, hence its use in the definition of the `f` property. The `parent` attribute is used to actually construct our path: by recursively retrieving parents, one constructs a path. You will notice that the definition of the `__lt__` method is missing - which brings us to the next question.

## Question 2

Later in the algorithm, we will feed the `AstarNode` into a priority queue (more specifically, Python's `heapq`). This will allow us to retrieve efficiently the node with the lower cost `f`). To this aim, we decide to implement a comparison operator for the node class.

a. implement the `__lt__` function above (note: this defines strict inequality `<`, not `<=`)

b. (answer after finishing the book) Should we (if so, why?) treat the case where $f_a = f_b$ but with $h_a < h_b$?

**<font color='red'>Given the same cost `f`, we should prefer the node with the lower `h` / higher `g` cost.**    </font>

**<font color='red'>Demonstration: there would be two possible situations if we kept exploring both paths $a$ and $b$ until the end of the algorithm:**
    </font>

**<font color='red'>1. One of the two paths is (stricly) better than the other. This information is however not available when we have to choose between $a$ and $b$, meaning that, at this point, choosing any path is correct.
</font>**

**<font color='red'>2. Both paths have the same final cost (i.e. they are alternative paths). In this case, if the number of iterations left is increasing in the value $h$ (fair assumption for most heuristics), then the path with the lower $h$ value has less "effort" left to reach the destination.
  </font>**


# Heuristic

The A* algorithm needs a heuristic `h` to direct the search. This function's goal is to give a rough estimate of the cost from a node `a` to a node `b` (usually the goal node). To allow trying out different heuristics, we created an abstract `AstarHeuristic` class:

In [6]:
from abc import ABC, abstractmethod

class AstarHeuristic(ABC):
    @abstractmethod
    def __call__(self, xs: int, ys: int, xf: int, yf: int) -> float:
        pass

Where `xs`, `ys` and `xf`, `yf` are the coordinate of the start and end node respectively.


## Question 3

The novel idea A* brought was the use of a heuristic. From this function's properties can various properties of A* be derived. Namely, an *optimistic* heuristic guarantees the algorithm's output will be optimal. Here, *optimistic* means the value given by the function is a lower-bound to the real cost of going from one node to another, i.e.:

$$
  h~\text{is optimistic } \Leftrightarrow h(n_a, n_b) \le g^*(n_a, n_b), \forall n_a, n_b \in \mathcal{N}
$$

Where $g^*(a, b)$ is the function giving the optimal cost for going from $a$ to $b$.


a. (answer after finishing the book) why would we ever want to use a non-optimistic heuristic?

b. (answer after finishing the book) are all optimistic heuristics equivalent?

c. Implement the heuristic below

In [7]:
class Norm1AstarHeuristic(AstarHeuristic):
    def __call__(self, xs: int, ys: int, xf: int, yf: int) -> float:
        return abs(xs - xf) + abs(ys - yf)

Looking at the `grass_fire.py` code, you may have noticed we associate to cost-to-go (i.e. the cost of going from a node A to a node B) to the destination node (i.e. we do not explictly create the edges). This association is done by passing a `cost_map` $c : (\mathbb{R} \times \mathbb{R}) \rightarrow  \mathbb{R}$ to the constructor of the graph.
This map returns the cost-to-go associated to the node (identified by its $(x, y)$ coordinates).

We do the same with our `AstarGraph`, except we create `AstarNode`s instead of `GrassFireNode`s. The rest of the implementation is missing some bits. These you have to write yourself.

In [34]:
from graph import Graph
from typing import List
from math import inf
from logging import info, warning

class AstarGraph(Graph[AstarNode]):
    def __init__(self, max_x: int, max_y: int, cost_map, h: AstarHeuristic) -> None:
        super().__init__(max_x, max_y)
        self.h = h
        for z in range(max_x * max_y):
            x, y = self._unflatten(z)
            self._nodes.append(AstarNode(x, y, cost_map[(x, y)]))
            
    def reset_nodes(self):
        for n in self._nodes:
            n.cost = inf
            n.parent = None

    def collect_path(self, node: AstarNode) -> List[AstarNode]:
        path = [node]
        while path[-1].parent is not None:
            path.append(path[-1].parent)
        path.reverse()
        return path
    
    def get_neighbors(self, node: AstarNode, make_8_connected=True) -> List[AstarNode]:
        out = []
        du = [
            (-1, 0),
            (1, 0),
            (0, -1),
            (0, 1),
        ]
        if make_8_connected:
            du += [
                (1, 1),
                (1, -1),
                (-1, 1),
                (-1, -1),
            ]
            
        # Note: this gives us an 8-connected grid
        for (dx, dy) in du:
            if dx == 0 and dy == 0:
                continue
            next_node = self.try_node(node.x + dx, node.y + dy)
            if next_node:
                out.append(next_node)
        return out

    def compute_path(self, xs: int, ys: int, xf: int, yf: int) -> List[AstarNode]:
        self.reset_nodes()
        start = self.node(xs, ys)
        start.cost = 0 # cost from start to itself is obviously 0
        heap = [start]

        max_iter = 1000000
        for n in range(max_iter):
            try:
                node_to_explore = heapq.heappop(heap)
            except IndexError:
                warning(
                    f"No path exists from start ({xs}, {ys}) to goal ({xf}, {yf}) after {n+1} iterations"
                )
                return []

            x = node_to_explore.x
            y = node_to_explore.y

            if x == xf and y == yf:
                info(
                    f"Found path from start ({xs}, {ys}) to goal ({xf}, {yf}) after {n+1} iterations and {n+1+len(heap)} nodes added"
                )
                return self.collect_path(node_to_explore)

            for next_node in self.get_neighbors(node_to_explore, True):
                # The cost to go is the Euclidian distance crossed in each node,
                # multiplied by the nodes' cost-to-go
                # We do Euclidian in case the grid is 8-connected (see get_neighbors function),
                # but for a 4-connected grid this changes nothing
                dx = next_node.x - node_to_explore.x
                dy = next_node.y - node_to_explore.y
                dg = (
                    (dx**2 + dy**2) ** 0.5
                    * (next_node.cost_to_go + node_to_explore.cost_to_go)
                    * 0.5
                )
                g = node_to_explore.cost + dg

                # Already had a better path to reach this node, do nothing
                if next_node.cost <= g:
                    continue

                next_node.cost = g
                next_node.parent = node_to_explore
                h = self.h(next_node.x, next_node.y, xf, yf)
                next_node.h = h
                heapq.heappush(heap, next_node)

        warning(f"Could not find a path after {max_iter} iterations")
        return []

a. Why do we check `x == xf and y == yf` after popping a node from the heap, rather than before inserting it (i.e. inside the `for` loop)?

# Trying out the solution

Let's try the same thing we did with the `GrassFire`, but with our new A*. This is a trivial test (since there are no obstacles), but gives us a good idea of the best-case complexity:

In [35]:
h = Norm1AstarHeuristic()
a = AstarGraph
jm.run_astar_with_heuristic({'grid_size': 5, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 10, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 20, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 50, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 100, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 200, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 400, 'number_obstacles': 0, 'no_render': True}, a, h)
jm.run_astar_with_heuristic({'grid_size': 500, 'number_obstacles': 0, 'no_render': False}, a, h)

Path search took 0.14862199986964697ms, final cost: 5.656854249492381
Path search took 0.40372500006924383ms, final cost: 12.727922061357859
Path search took 1.072488000318117ms, final cost: 26.870057685088817
Path search took 4.938817000038398ms, final cost: 69.2964645562816
Path search took 16.85031600027287ms, final cost: 140.00714267493635
Path search took 81.26730700041662ms, final cost: 281.428498912247
Path search took 331.2316130004547ms, final cost: 564.2712113868682
Path search took 527.9987189996973ms, final cost: 705.6925676241789


#### Footnotes

[1] A graph and a graph search algorithm are two different things, and the latter can be implemented generically over the former. This requires however the Graph (as well as the Node and Edge) class to implement a number of functions. While not a specifically daunting task, this is not within the scope of this exercise. We have therefore decided to keep the graph creation and search tighly coupled, allowing you (almost) complete freedom over the way nodes and edges are stored. For an example implementation of a generic graph library, the Boost Graph Library (BGL) is your best bet: https://www.boost.org/doc/libs/1_81_0/libs/graph/doc/index.html